<a href="https://colab.research.google.com/github/robbytbg/classWork/blob/main/BDO4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#full blind search method
fsearch=function(search, FUN, type="min",...)
{ 		x=apply(search,1,FUN,...)
# run FUN over all search rows
ib=switch(type, min=which.min(x),max=which.max(x))
return(list(index=ib, sol=search[ib,],eval=x[ib]))
}

#grid search method
gsearch=function(step,lower,upper,FUN,type="min",...)
{
  D=length(step) 			# dimension
  domain=vector("list",D)		 # domain values
  L=vector(length=D)			 # auxiliary vector
  for(i in 1:D)
  {
    domain[[i]]=seq(lower[i],upper[i],by=step[i])
    L[i]=length(domain[[i]])
  }
  LS=prod(L)
  s=matrix(ncol=D,nrow=LS) 	# set the search space
  for(i in 1:D)
  {
    if(i==1) E=1
    else E=E*L[i-1]
    s[,i]=rep(domain[[i]],length.out=LS,each=E)
  }
  fsearch(s,FUN,type,...)			 # best solution
}

#sales function
sales=function(x, A=1000,B=200,C=141, m=seq(2,length.out=length(x),by=-0.25))
{
  return(round(m*(A/log(x+B)-C),digits=0))
}

#cost function
cost=function(units,A=100,cpu=35-5*(1:length(units)))
{
  return(A+cpu*units)
}

#profit function
profit=function(x) 		#x-a vector of prices
{ x=round(x,digits=0) 	# convert x into integer
s=sales(x) 			# get the expected sales
c=cost(s) 			# get the expected cost
profit=sum(s*x-c)		 # compute the profit
return(profit)
# local variables x, s, c and profit are lost from here
}

#bag prices problem using grid dearch
PTM=proc.time() 			# start clock
S1=gsearch(rep(100,5),rep(1,5),rep(1000,5),profit,"max")
sec=(proc.time()-PTM)[3] 	# get seconds elapsed
cat("gsearch best s:",S1$sol,"f:",S1$eval,"time:",sec,"s\n")

gsearch best s: 401 401 401 401 501 f: 43142 time: 2.004 s


In [2]:
#full blind search method
fsearch=function(search, FUN, type="min",...)
{ 		x=apply(search,1,FUN,...)
# run FUN over all search rows
ib=switch(type, min=which.min(x),max=which.max(x))
return(list(index=ib, sol=search[ib,],eval=x[ib]))
}

#grid search method
gsearch=function(step,lower,upper,FUN,type="min",...)
{
  D=length(step) 			# dimension
  domain=vector("list",D)		 # domain values
  L=vector(length=D)			 # auxiliary vector
  for(i in 1:D)
  {
    domain[[i]]=seq(lower[i],upper[i],by=step[i])
    L[i]=length(domain[[i]])
  }
  LS=prod(L)
  s=matrix(ncol=D,nrow=LS) 	# set the search space
  for(i in 1:D)
  {
    if(i==1) E=1
    else E=E*L[i-1]
    s[,i]=rep(domain[[i]],length.out=LS,each=E)
  }
  fsearch(s,FUN,type,...)			 # best solution
}


# real-value function: sphere
sphere=function(x) sum(x^2)
cat("sphere:\n") 				# D=2, easy task

#sphere problem using grid search
S=gsearch(rep(1.1,2),rep(-5.2,2),rep(5.2,2),sphere,"min")
cat("gsearch s:",S$sol,"f:",S$eval,"\n")

sphere:
gsearch s: 0.3 0.3 f: 0.18 


In [3]:
#nested grid search
ngsearch=function(levels,step,lower,upper,FUN,type,...)
{
  stop=FALSE;
  i=1 			# auxiliary objects
  bcur=switch(type,min=list(sol=NULL,eval=Inf),max=list(sol=NULL,eval=-Inf))

  while(!stop) 		# cycle while stopping criteria is not met
  {
     s=gsearch(step,lower,upper,FUN,type,...)
    # if needed, update best current solution:

    if((type=="min" && s$eval<bcur$eval)||(type=="max" && s$eval > bcur$eval))
      bcur=s
    if(i<levels) 			#update step, lower and upper
    {
        step=step/2
        interval=(upper-lower)/4
        lower=sapply(lower,max,s$sol-interval)
        upper=sapply(upper,min,s$sol+interval)
    }
    if((i>=levels)||(sum((upper-lower)<=step)>0)) stop=TRUE
    else i=i+1
  }
  return(bcur) 			# best solution
}


#sphere problem using nested grid search
S=ngsearch(3,rep(3,2),rep(-5.2,2),rep(5.2,2),sphere,"min")
cat("ngsearch s:",S$sol,"f:",S$eval,"\n")

ngsearch s: -0.1 -0.1 f: 0.02 


In [4]:
# real-value functions: rastrigin
rastrigin=function(x) 10*length(x)+sum(x^2-10*cos(2*pi*x))

#Rastrigin problem using grid search
#load full blind search method
#load grid search method
cat("rastrigin:\n") 				# D=2, easy task

S=gsearch(rep(1.1,2),rep(-5.2,2),rep(5.2,2),rastrigin,"min")
cat("gsearch s:",S$sol,"f:",S$eval,"\n")

rastrigin:
gsearch s: -1.9 -1.9 f: 11.03966 


In [5]:
#Rastrigin problem using nested grid search
#load full blind search method
#load grid search method
#load nested grid search method
S=ngsearch(3,rep(3,2),rep(-5.2,2),rep(5.2,2),rastrigin,"min")
cat("ngsearch s:",S$sol,"f:",S$eval,"\n")

ngsearch s: -0.1 -0.1 f: 3.83966 
